# 2人の意思決定（石取りゲームの学習）<br>
# Decison-Making by Two People (Learning of Stone Picking Game)<br>

・強化学習の1つであるQラーニングを用いる<br>
・This time, we use Q-learning, which is one of the reinforcement learning.<br>
<br>
・エージェントは2つ用いる<br>
・We use two agents.


In [46]:
import numpy as np
import math

In [26]:
number_color = 4
number_space = 3

In [13]:
def get_HB(word, answer):
    word_left, answer_left = [], []
    hit, blow = 0, 0
    
    # Judge hit
    for word_i in range(len(answer)):
        if word[word_i] == answer[word_i]:
            hit += 1
        else:
            word_left.append(word[word_i])
            answer_left.append(answer[word_i])
    
    # Judge blow
    for word_i in range(len(word_left)):
        if word_left[word_i] in answer_left:
            blow += 1
            answer_left.remove(word_left[word_i])
            
    return [hit, blow]
    

In [16]:
# get all potential wors list
def get_potential_list(potential_list, request, responce):
    new_potential_list = []
    for potential in potential_list:
        if get_HB(potential, request) == responce:
            new_potential_list.append(potential)
    return new_potential_list
    

In [50]:
def calculate_info(expect_HB_row):
    info = 0
    sum_row = np.sum(expect_HB_row)
    for item in expect_HB_row:
        if item != 0:
            info += item / sum_row * math.log2(item)
    return info

In [78]:
def expected_info(potential_list, original_list, number_space):
    info = np.zeros((len(original_list)))
    for original_i in range(len(original_list)):
        expect_HB = np.zeros(((number_space + 1) ** 2))
        for potential_i in range(len(potential_list)):
            HB = get_HB(original_list[original_i], potential_list[potential_i])
            expect_HB[HB[0] * number_space + HB[1]] += 1
        info[original_i] = calculate_info(expect_HB)
    return info

In [79]:
def make_original_list(number_color, number_space):
    original_list = []
    for i in range(number_color ** number_space):
        original_list.append(np.base_repr(i, number_color).zfill(number_space))
    return original_list

In [117]:
def add_new_info(info, info_list):
    contain_TF = True
    for item in info:
        if item not in info_list:
            info_list.append(item)
            contain_TF = False
    #if contain_TF == False:
        #print("Add new item!!")
    return info_list, contain_TF
    

In [118]:
def get_InfoList(number_color, number_space):
    original_list = make_original_list(number_color, number_space)
    info_list = []
    contain_count = 0
    i = 0
    while contain_count < 10 * (number_color ** number_space):
        HB = [0, 0]
        answer = np.random.choice(original_list)
        #print("Answer: ", answer)
        potential_list = original_list.copy()
        current_containTF = True
        while HB != [number_space, 0]:
            request = np.random.choice(original_list)
            #print("Request: ", request)
            HB = get_HB(request, answer)
            #print("HB: ", HB)
            potential_list = get_potential_list(potential_list, request, HB)
            #print("Potential list: ", potential_list)
            info = expected_info(potential_list, original_list, number_space)
            info_list, containTF = add_new_info(info, info_list)
            if (current_containTF == True) and (containTF == False):
                current_containTF = False
        if (current_containTF == True):
            contain_count += 1
        else:
            contain_count = 0
        i += 1
    
    info_list = np.sort(info_list)
    return info_list

In [ ]:
def get_ChangeList(number_color, number_space, info_list):
    original_list = make_original_list(number_color, number_space)
    change_list = np.zeros((len(info_list), number_color ** number_space))
    contain_count = 0

    while contain_count < 50 * (number_color ** number_space):
        HB = [0, 0]
        answer = np.random.choice(original_list)
        #print("Answer: ", answer)
        potential_list = original_list.copy()
        current_containTF = True
        while HB != [number_space, 0]:
            request = np.random.choice(original_list)
            #print("Request: ", request)
            HB = get_HB(request, answer)
            #print("HB: ", HB)
            potential_list = get_potential_list(potential_list, request, HB)
            #print("Potential list: ", potential_list)
            info = expected_info(potential_list, original_list, number_space)
            info_list, containTF = add_new_info(info, info_list)
            if (current_containTF == True) and (containTF == False):
                current_containTF = False
        if (current_containTF == True):
            contain_count += 1
        else:
            contain_count = 0
    return change_list

In [119]:
info_list = get_InfoList(number_color, number_space)

In [120]:
print(info_list)

[0.         0.25       0.28571429 0.33333333 0.4        0.5
 0.57142857 0.59436094 0.6        0.66666667 0.67548875 0.67926964
 0.75       0.75054306 0.79248125 0.8        0.83333333 0.84436094
 0.85714286 0.87548875 0.88888889 0.89624063 0.9509775  0.96498393
 0.97276528 1.         1.         1.05664167 1.06290729 1.06290729
 1.07548875 1.09436094 1.10784107 1.10784107 1.11111111 1.12581458
 1.14285714 1.14285714 1.1509775  1.16666667 1.16666667 1.18872188
 1.1949875  1.19677768 1.2        1.2        1.22087574 1.22957396
 1.25       1.25069821 1.26528088 1.27548875 1.27886389 1.30461875
 1.33333333 1.33333333 1.3509775  1.3509775  1.35853929 1.38292794
 1.39624063 1.4        1.41176471 1.41720972 1.42857143 1.43320753
 1.433985   1.43872188 1.45460914 1.45616985 1.45616985 1.45914792
 1.46666667 1.49096853 1.5        1.51218227 1.5364125  1.5364125
 1.54498015 1.56096405 1.57397603 1.5849625  1.5849625  1.59033304
 1.59436094 1.6        1.61822206 1.62581458 1.63943194 1.63943194
 1.

In [121]:
len(info_list)

167

変数の設定 Settings of variables

In [2]:
BOTTLE_N = 11#石の数 the number of stones

#Q値の初期化 Q-value initialization
QV0=np.zeros((BOTTLE_N+1,3), dtype=np.float32)
QV1=np.zeros((BOTTLE_N+1,3), dtype=np.float32)
QVs = [QV0, QV1]

状態遷移：状態を変化させるための関数<br>
State transiton: The function used to change state

In [3]:
def step(action, state, turn):
    state = state + action + 1
    rewards = [0,0]
    done = False
    if (state>=BOTTLE_N):
        state = BOTTLE_N
        rewards[turn] = -1
        rewards[(turn+1)%2] = 1
        done = True
    return state, rewards, done

行動選択：Q値から次の行動を選択するための関数<br>
Select actions: The function to select next action from Q-value

In [4]:
def getAction(state, epsilon, qv):
    #徐々に最適行動のみをとる、ε-greedy法
    #Gradually take only the optimal action at that time, ε-greedy method
    if epsilon > np.random.uniform(0, 1):
        next_action = np.random.choice([0, 1, 2])
    else:
        a = np.where(qv[state]==qv[state].max())[0]
        #その時点での最適行動が複数ある場合はランダムに選択
        #If there are multiple optimal actions at that time, select randomly from them
        next_action = np.random.choice(a)
    return next_action

Q値の更新：状態，行動，報酬，次の状態を用いてQ値を更新するための関数<br>
Update Q-value: The function to update Q-value based on the current state, the behavior, the reward and the next state

In [5]:
def updateQValue(action, reward, state, state_old, qv, gamma, alpha):
    maxQ = np.max(qv[state])
    qv[state_old][action] = (1-alpha)*qv[state_old][action]+alpha*(reward + gamma*maxQ);

変数の設定 Settings of variables

In [6]:
num_episodes = 100#エピソード数 number of episodes
gamma = 0.9#割引率 discount rate
alpha = 0.5#学習率 learning rate

強化学習の実行 Do reinforcement learning

In [7]:
for episode in range(num_episodes):  #試行数分繰り返す
    state = 0
    state_old = [0,0]
    rewards = [0,0]
    actions = [0,0]
    epsilon = 0.5 * (1 / (episode + 1))
    while(1):
        actions[0] = getAction(state, epsilon, QVs[0])
        state_old[0] = state
        state, rewards, done = step(state, actions[0], 0)
        updateQValue(actions[1], rewards[1], state, state_old[1], QVs[1], gamma, alpha)
        if (done==True):
            updateQValue(actions[0], rewards[0], state, state_old[0], QVs[0], gamma, alpha)
            print('{} : 0 Lose, 1 Win!!'.format(episode))
            break
        actions[1] = getAction(state, epsilon, QVs[1])
        state_old[1] = state
        state, rewards, done = step(state, actions[1], 1)
        updateQValue(actions[0], rewards[0], state, state_old[0], QVs[0], gamma, alpha)
        if (done==True):
            updateQValue(actions[1], rewards[1], state, state_old[1], QVs[1], gamma, alpha)
            print('{} : 0 Win!!, 1 Lose'.format(episode))
            break

0 : 0 Win!!, 1 Lose
1 : 0 Lose, 1 Win!!
2 : 0 Win!!, 1 Lose
3 : 0 Lose, 1 Win!!
4 : 0 Win!!, 1 Lose
5 : 0 Lose, 1 Win!!
6 : 0 Lose, 1 Win!!
7 : 0 Lose, 1 Win!!
8 : 0 Lose, 1 Win!!
9 : 0 Lose, 1 Win!!
10 : 0 Win!!, 1 Lose
11 : 0 Lose, 1 Win!!
12 : 0 Win!!, 1 Lose
13 : 0 Win!!, 1 Lose
14 : 0 Win!!, 1 Lose
15 : 0 Lose, 1 Win!!
16 : 0 Win!!, 1 Lose
17 : 0 Win!!, 1 Lose
18 : 0 Lose, 1 Win!!
19 : 0 Win!!, 1 Lose
20 : 0 Win!!, 1 Lose
21 : 0 Win!!, 1 Lose
22 : 0 Lose, 1 Win!!
23 : 0 Lose, 1 Win!!
24 : 0 Lose, 1 Win!!
25 : 0 Lose, 1 Win!!
26 : 0 Lose, 1 Win!!
27 : 0 Win!!, 1 Lose
28 : 0 Win!!, 1 Lose
29 : 0 Win!!, 1 Lose
30 : 0 Win!!, 1 Lose
31 : 0 Win!!, 1 Lose
32 : 0 Win!!, 1 Lose
33 : 0 Lose, 1 Win!!
34 : 0 Lose, 1 Win!!
35 : 0 Win!!, 1 Lose
36 : 0 Win!!, 1 Lose
37 : 0 Lose, 1 Win!!
38 : 0 Lose, 1 Win!!
39 : 0 Win!!, 1 Lose
40 : 0 Win!!, 1 Lose
41 : 0 Win!!, 1 Lose
42 : 0 Win!!, 1 Lose
43 : 0 Win!!, 1 Lose
44 : 0 Win!!, 1 Lose
45 : 0 Win!!, 1 Lose
46 : 0 Win!!, 1 Lose
47 : 0 Win!!, 1 Lose
48

In [8]:
print("Agent 0")
print(QVs[0])
print("Agent 1")
print(QVs[1])
np.savetxt('QV0.txt', QVs[0])
np.savetxt('QV1.txt', QVs[1])

Agent 0
[[ 0.          0.80999875 -0.05254638]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.16699216  0.          0.89999926]
 [-0.21926622  0.8999994   0.        ]
 [ 0.8999994   0.         -0.39374998]
 [-0.66772705 -0.7246094  -0.65937495]
 [ 0.          0.          1.        ]
 [-0.21249999  1.         -0.5       ]
 [ 1.         -0.5        -0.5       ]
 [-0.96875    -0.96875    -0.984375  ]
 [ 0.          0.          0.        ]]
Agent 1
[[-0.72898775  0.          0.        ]
 [ 0.          0.          0.        ]
 [-0.80999213 -0.80999446 -0.80999273]
 [ 0.          0.          0.50689083]
 [-0.17718749  0.64335936  0.        ]
 [ 0.590625   -0.39374998 -0.4359375 ]
 [-0.8999995  -0.89999956 -0.89999944]
 [-0.12851562 -0.325       0.875     ]
 [-0.38124996  0.96875    -0.5       ]
 [ 0.99609375  0.         -0.75      ]
 [-1.         -1.         -1.        ]
 [ 0.          0.          0.        ]]


必勝法と同じ取り方を学習しているかの確認<br>
Check if you are learning the same winning method.

In [9]:
for j in range(2):
    print("Agent", j)
    for i in range(BOTTLE_N):
        a = np.where(QVs[j][i]==QVs[j][i].max())[0]
        print('残り本数',BOTTLE_N-i,'取る数',a+1,'必勝法',(BOTTLE_N-i-1)%4,'なんでもよい' if (BOTTLE_N-i-1)%4 == 0 else \
              ('不定' if a.size >1 else ('正解' if (BOTTLE_N-i-1)%4 == a+1 else '不正解')))

Agent 0
残り本数 11 取る数 [2] 必勝法 2 正解
残り本数 10 取る数 [1 2 3] 必勝法 1 不定
残り本数 9 取る数 [1 2 3] 必勝法 0 なんでもよい
残り本数 8 取る数 [3] 必勝法 3 正解
残り本数 7 取る数 [2] 必勝法 2 正解
残り本数 6 取る数 [1] 必勝法 1 正解
残り本数 5 取る数 [3] 必勝法 0 なんでもよい
残り本数 4 取る数 [3] 必勝法 3 正解
残り本数 3 取る数 [2] 必勝法 2 正解
残り本数 2 取る数 [1] 必勝法 1 正解
残り本数 1 取る数 [1 2] 必勝法 0 なんでもよい
Agent 1
残り本数 11 取る数 [2 3] 必勝法 2 不定
残り本数 10 取る数 [1 2 3] 必勝法 1 不定
残り本数 9 取る数 [1] 必勝法 0 なんでもよい
残り本数 8 取る数 [3] 必勝法 3 正解
残り本数 7 取る数 [2] 必勝法 2 正解
残り本数 6 取る数 [1] 必勝法 1 正解
残り本数 5 取る数 [3] 必勝法 0 なんでもよい
残り本数 4 取る数 [3] 必勝法 3 正解
残り本数 3 取る数 [2] 必勝法 2 正解
残り本数 2 取る数 [1] 必勝法 1 正解
残り本数 1 取る数 [1 2 3] 必勝法 0 なんでもよい
